In [1]:
import pandas as pd
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import seaborn as sns

### Train

In [2]:
train = pd.read_csv('../../ml-data/cyberattack_detection/3_selected_features/A1_train_data_VT.csv', sep=',')

In [3]:
train.head()

,TotalPackets,TotalBytes,SourceBytes,PacketsSec,TotalBytesSec,SourceBytesSec,Protocol,SrcIPAddress,SrcPort,DestIPAddress,DestPort,State,DestTypeService
0,-0.008024,-0.008615,-0.003897,-0.296301,-0.308511,-0.141071,11.0,5663.0,123467.0,475015.0,57944.0,120.0,0.0
1,-0.010103,-0.008919,-0.004151,-0.296351,-0.308520,-0.141076,11.0,5663.0,123658.0,46220.0,34823.0,327.0,4.0
2,-0.010363,-0.008918,-0.004149,-0.296351,-0.308520,-0.141076,11.0,5663.0,123546.0,76319.0,34823.0,327.0,4.0
3,-0.010363,-0.008917,-0.004151,-0.296351,-0.308520,-0.141076,11.0,5663.0,123606.0,452301.0,34823.0,327.0,4.0
4,-0.010623,-0.008919,-0.004149,-0.296351,-0.308520,-0.141076,11.0,5663.0,123487.0,79211.0,34823.0,327.0,4.0


In [4]:
# set isolation = 0.02
isolation_forest = IsolationForest(contamination=0.02)

In [5]:
isolation_forest.fit(train)

IsolationForest(contamination=0.02)

### Test

In [6]:
test = pd.read_csv('../../ml-data/cyberattack_detection/3_selected_features/A1_test_data_VT.csv', sep=',')

In [7]:
test_predict = isolation_forest.predict(test)
decision_function = isolation_forest.decision_function(test)

In [8]:
test['anomaly'] = test_predict
test['score'] = decision_function

In [9]:
test['anomaly'].value_counts()

 1    1030449
-1      23396
Name: anomaly, dtype: int64

In [ ]:
sns.pairplot(test, hue = 'anomaly')

In [11]:
anomalies = test.loc[test['anomaly'] == -1]

In [12]:
anomalies.head()

,TotalPackets,TotalBytes,SourceBytes,PacketsSec,TotalBytesSec,SourceBytesSec,Protocol,SrcIPAddress,SrcPort,DestIPAddress,DestPort,State,DestTypeService,anomaly,score
29,0.177809,0.008249,0.022155,-0.296300,-0.308511,-0.141067,11.0,9456.0,125131.0,106313.0,44531.0,178.0,0.0,-1,-0.029751
35,0.092560,0.005278,0.018370,-0.296303,-0.308512,-0.141068,11.0,7374.0,46840.0,631365.0,34666.0,189.0,0.0,-1,-0.039965
39,0.132066,0.007298,0.010765,-0.296301,-0.308511,-0.141071,11.0,5363.0,130150.0,241570.0,35739.0,189.0,0.0,-1,-0.010192
249,0.386255,0.306761,0.016051,-0.260652,0.008747,-0.128602,11.0,3047.0,46659.0,497552.0,83669.0,120.0,0.0,-1,-0.113598
1829,0.172871,0.008967,0.021952,-0.296300,-0.308510,-0.141067,11.0,9456.0,125131.0,106313.0,44531.0,178.0,0.0,-1,-0.029751


In [13]:
original_test = pd.read_csv('../../ml-data/cyberattack_detection/1_raw_data/A2_1/clean_test_data', sep=',')

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
anomalies_orig_feat = original_test[original_test.index.isin(anomalies.index)]

In [15]:
anomalies_orig_feat = anomalies_orig_feat.merge(anomalies[['score']], left_index = True, right_index=True)

In [16]:
anomalies_orig_feat.head(10)

,Timestamp,Duration,Protocol,SrcIPAddress,SrcPort,Direction,DestIPAddress,DestPort,State,SrcTypeService,DestTypeService,TotalPackets,TotalBytes,SourceBytes,PacketsSec,TotalBytesSec,SourceBytesSec,score
29,2012/12/01 13:11:28.851831,3730.155481,tcp,223.123.73.31,49160,->,138.29.91.235,22,PA_A,0.0,0.0,726,65410,43750,0.194630,1.753546e+01,11.728734,-0.029751
35,2012/12/01 13:12:28.147391,3470.644866,tcp,200.231.81.55,1049,->,230.72.91.222,13363,PA_PA,0.0,0.0,398,54101,37464,0.114676,1.558817e+01,10.794536,-0.039965
39,2012/12/01 13:12:41.413006,3490.098237,tcp,178.50.91.161,51444,<->,162.183.3.133,143,PA_PA,0.0,0.0,550,61790,24832,0.157589,1.770437e+01,7.114986,-0.010192
249,2012/12/01 13:34:30.899435,1.850078,tcp,153.217.92.157,1040,->,207.177.143.150,5678,FSPA_FSPA,0.0,0.0,1528,1201775,33613,825.911124,6.495807e+05,18168.423169,-0.113598
1829,2012/12/01 14:11:29.044873,3664.463491,tcp,223.123.73.31,49160,->,138.29.91.235,22,PA_A,0.0,0.0,707,68145,43414,0.192934,1.859617e+01,11.847300,-0.029751
1871,2012/12/01 14:12:29.621949,3508.240697,tcp,136.87.165.89,12541,->,230.72.91.222,13363,PA_PA,0.0,0.0,176,21103,14178,0.050168,6.015266e+00,4.041342,-0.014097
1873,2012/12/01 14:12:35.989767,3582.235642,tcp,200.231.81.55,1049,->,230.72.91.222,13363,PA_PA,0.0,0.0,366,53766,33783,0.102171,1.500906e+01,9.430703,-0.039965
1909,2012/12/01 14:13:01.598907,3639.017926,tcp,178.50.91.161,51444,<->,162.183.3.133,143,PA_PA,0.0,0.0,518,59351,23754,0.142346,1.630962e+01,6.527585,-0.006482
1969,2012/12/01 14:14:22.366222,0.653870,tcp,132.200.95.181,1030,->,204.136.12.18,80,SRPA_FSPA,0.0,0.0,850,817410,18766,1299.952590,1.250111e+06,28699.894474,-0.134337
2076,2012/12/01 14:15:39.871327,14.729237,tcp,132.200.95.181,1066,->,191.117.240.239,80,SRPA_FSPA,0.0,0.0,240,203414,7596,16.294123,1.381022e+04,515.708994,-0.036463


In [17]:
import numpy as np
anomalies_orig_feat.loc[anomalies_orig_feat['PacketsSec'] != np.inf].describe().apply(lambda s: s.apply(lambda x: format(x, 'g')))

,Duration,SrcTypeService,DestTypeService,TotalPackets,TotalBytes,SourceBytes,PacketsSec,TotalBytesSec,SourceBytesSec,score
count,23375,23367,3097,23375,23375,23375,23375,23375,23375,23375
mean,835.715,0,0,1234.91,1.03965e+06,870799,3224.26,235328,110924,-0.0492772
std,589.737,0,0,27774,1.42755e+07,1.35019e+07,32553.5,2.4476e+06,1.21165e+06,0.0299593
min,4e-06,0,0,1,57,0,0.000933159,0.0836732,0,-0.188221
25%,352.798,0,0,14,13744,13005.5,0.0156601,15.246,14.9383,-0.0568471
50%,950.894,0,0,23,23332,21140,0.034824,35.5004,35.243,-0.045956
75%,1097.69,0,0,37,38233,35649.5,0.158925,140.163,80.8843,-0.0332698
max,3858.04,0,0,3.68056e+06,9.45069e+08,9.83047e+08,1e+06,8.475e+07,4.492e+07,-1.13625e-06


In [18]:
anomalies_orig_feat['SrcIPAddress'].nunique()

149

In [19]:
anomalies_orig_feat.to_csv('../4 Results/A1_test_iForest0.02_VT_orig_feat.csv', sep=',')

End